# A full training

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [36]:
# !pip install datasets evaluate transformers[sentencepiece]
# !pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

Now <span style="color:blue">we’ll see how to achieve the same results as we did in the last section <b>without using the `Trainer` class</b></span>. Again, we assume you have done the data processing in section 2. Here is a short summary covering everything you will need:

In [37]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Found cached dataset glue (/Users/prasanth.thangavel/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/prasanth.thangavel/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-bb21e6423b980722.arrow
Loading cached processed dataset at /Users/prasanth.thangavel/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d1e8c90b5d349f7a.arrow
Loading cached processed dataset at /Users/prasanth.thangavel/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-33304e37c309912f.arrow


## Prepare for training

<img src="images/sketch-of-training-loop.png" style="width:650px;" title="sketch-of-training-loop">

Before actually writing our training loop, we will need to define a few objects. The first ones are the dataloaders we will use to iterate over batches. But before we can define those dataloaders, we need to apply a bit of postprocessing to our `tokenized_datasets`, to take care of some things that the `Trainer` did for us automatically. Specifically, we need to:

- Remove the columns corresponding to values the model does not expect (like the `sentence1` and `sentence2` columns).
- Rename the column `label` to `labels` (because the model expects the argument to be named `labels`).
- Set the format of the datasets so they return PyTorch tensors instead of lists.

Our `tokenized_datasets` has one method for each of those steps:

In [38]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

We can then check that the result only has columns that our model will accept:
```["attention_mask", "input_ids", "labels", "token_type_ids"]``` (in random order0)

Now that this is done, we can easily define our dataloaders:

In [39]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

To quickly check there is no mistake in the data processing, we can inspect a batch like this:

In [40]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 68]),
 'token_type_ids': torch.Size([8, 68]),
 'attention_mask': torch.Size([8, 68])}

Note that the actual shapes will probably be slightly different for you since we set `shuffle=True` for the training dataloader and we are padding to the maximum length inside the batch.

Now that we’re completely finished with data preprocessing (a satisfying yet elusive goal for any ML practitioner), let’s turn to the model. We instantiate it exactly as we did in the previous section:

In [41]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

To make sure that everything will go smoothly during training, we pass our batch to this model:

In [42]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

outputs

tensor(0.8332, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


SequenceClassifierOutput(loss=tensor(0.8332, grad_fn=<NllLossBackward0>), logits=tensor([[0.5475, 0.0618],
        [0.5653, 0.0534],
        [0.5560, 0.0761],
        [0.5580, 0.0787],
        [0.5530, 0.0723],
        [0.5288, 0.0811],
        [0.5002, 0.1398],
        [0.5302, 0.1002]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

<span style="color:blue">All 🤗 Transformers models will return the loss when labels are provided, and we also get the logits </span>(two for each input in our batch, so a tensor of size 8 x 2).

<span style="color:blue">We’re almost ready to write our training loop! We’re just missing two things: <b>an optimizer</b> and a <b>learning rate scheduler</b></span>. Since we are trying to replicate what the `Trainer` was doing by hand, we will use the same defaults. The optimizer used by the `Trainer` is `AdamW`, which is the same as Adam, but with a twist for weight decay regularization (see [“Decoupled Weight Decay Regularization”](https://arxiv.org/abs/1711.05101) by Ilya Loshchilov and Frank Hutter):

In [43]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/Users/prasanth.thangavel/.pyenv/versions/3.10.5/envs/fastai_related/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Finally, <span style="color:blue">the learning rate scheduler used by default is just a <b>progressive linear decay</b> from the maximum value (5e-5) to 0,</span>. To properly define it, we need to know the number of training steps we will take, which is the number of epochs we want to run multiplied by the number of training batches (which is the length of our training dataloader). The `Trainer` uses three epochs by default, so we will follow that:

<img src="images/learning-rate-scheduler.png" style="width:850px;" title="learning-rate-scheduler">

In [44]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


## The training loop

One last thing: we will want to use the GPU if we have access to one (on a CPU, training might take several hours instead of a couple of minutes). To do this, we define a `device` we will put our model and our batches on:

In [45]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cpu')

We are now ready to do the custom training loop! To get some sense of when training will be finished, we add a progress bar over our number of training steps, using the tqdm library:



In [46]:
# Takes ~15 mins Mac M1 Pro chip
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train() # set the model to training mode (e.g., enables features like dropout, batch norm, etc.)
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()} # move the training batch to GPU (if available)
        outputs = model(**batch) # forward pass through model (i.e., processed input batch and return output) 
        loss = outputs.loss # compute loss 
        loss.backward() # perform backpropagation to compute gradients

        optimizer.step() # update model params (using computed gradients)
        lr_scheduler.step() # update learning rate according to its schedule
        optimizer.zero_grad() # reset gradients to zero to prevent accumulation across multiple iterations.

        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

You can see that the core of the training loop looks a lot like the one in the introduction. We didn’t ask for any reporting, so this training loop will not tell us anything about how the model fares. We need to add an evaluation loop for that.

## The evaluation loop

As we did earlier, we will use a metric provided by the 🤗 Evaluate library. We’ve already seen the `metric.compute()` method, but <span style="color:blue">metrics can actually accumulate batches for us as we go over the prediction loop with the method `add_batch()`</span>. Once we have accumulated all the batches, we can get the final result with `metric.compute()`. Here’s how to implement all of this in an evaluation loop:

In [47]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval() # set the model to evaluation mode => Disable dropout and batch norm layers
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad(): # Disable gradient computation (as not needed for eval)
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.875, 'f1': 0.9131175468483815}

Again, your results will be slightly different because of the randomness in the model head initialization and the data shuffling, but they should be in the same ballpark.

> ✏️ Try it out! Modify the previous training loop to fine-tune your model on the SST-2 dataset.

## Supercharge your training loop with 🤗 Accelerate

The training loop we defined earlier works fine on a single CPU or GPU. But using the [🤗 Accelerate](https://github.com/huggingface/accelerate) library, with just a few adjustments we can enable distributed training on multiple GPUs or TPUs.

<img src="images/multiple-setups-for-training.png" style="width:650px;" title="multiple-setups-for-training">

<b>Starting from the creation of the training and validation dataloaders, here is what our full manual training loop code looks like</b>:

In [48]:
# Takes ~15 mins Mac M1 Pro chip, whereas ~3 hrs in free version of google colab
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

  0%|          | 0/1377 [00:00<?, ?it/s]

And here are the changes:

```python
+ from accelerate import Accelerator
  from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

+ accelerator = Accelerator()

  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
  optimizer = AdamW(model.parameters(), lr=3e-5)

- device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
- model.to(device)

+ train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
+     train_dataloader, eval_dataloader, model, optimizer
+ )

  num_epochs = 3
  num_training_steps = num_epochs * len(train_dataloader)
  lr_scheduler = get_scheduler(
      "linear",
      optimizer=optimizer,
      num_warmup_steps=0,
      num_training_steps=num_training_steps
  )

  progress_bar = tqdm(range(num_training_steps))

  model.train()
  for epoch in range(num_epochs):
      for batch in train_dataloader:
-         batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(**batch)
          loss = outputs.loss
-         loss.backward()
+         accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
```                              

The first line to add is the import line. The second line instantiates an `Accelerator` object that will look at the environment and initialize the proper distributed setup. 🤗 Accelerate handles the device placement for you, so you can remove the lines that put the model on the device (or, if you prefer, change them to use `accelerator.device` instead of `device`).

Then the main bulk of the work is done in the line that sends the dataloaders, the model, and the optimizer to `accelerator.prepare()`. This will wrap those objects in the proper container to make sure your distributed training works as intended. The remaining changes to make are removing the line that puts the batch on the device (again, if you want to keep this you can just change it to use `accelerator.device`) and replacing `loss.backward()` with `accelerator.backward(loss)`.

> ⚠️ In order to benefit from the speed-up offered by Cloud TPUs, we recommend padding your samples to a fixed length with the `padding="max_length"` and `max_length` arguments of the tokenizer.
If you’d like to copy and paste it to play around, here’s what the complete training loop looks like with 🤗 Accelerate:

If you’d like to copy and paste it to play around, here’s what the complete training loop looks like with 🤗 Accelerate:

In [49]:
# Takes ~5.2 mins with Mac M1 Pro chip => 3x faster using GPU
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

  0%|          | 0/1377 [00:00<?, ?it/s]

- <span style="color:green">Takes ~5.2 mins with Mac M1 Pro chip => <b>3x faster using GPU, instead of CPU in Mac</b></span>.
- <span style="color:green">Takes 2.8 mins with Amazon SageMaker GPU Tesla T4 (16GB) => <b>~6x faster using Sagemaker GPU, instead of Mac M1 Chip CPU. ~2x faster using Sagemaker GPU vs. Mac M1 Chip GPU</b></span>.
    - Running it with accelerator does not help as Sagemaker has only 1 GPU card

MAC M1 chip GPU usage details while training with `Accelerator`: <img src="images/mac-m1-chip-gpu-usage.png" style="width:150px;" title="mac-m1-chip-gpu-usage">

AWS sagemaker GPU details while training: <img src="images/nvidia-smi-amazon-sagemaker.png" style="width:650px;" title="nvidia-smi-amazon-sagemaker">

Putting this in a `train.py` script will make that script runnable on any kind of distributed setup. To try it out in your distributed setup, run the command:

```accelerate config```

which will prompt you to answer a few questions and dump your answers in a configuration file used by this command:

```accelerate launch train.py```

which will launch the distributed training.

<img src="images/accelerate-training.png" style="width:650px;" title="accelerate-training">

If you want to try this in a Notebook (for instance, to test it with TPUs on Colab), just paste the code in a `training_function()` and run a last cell with:

In [50]:
from accelerate import notebook_launcher

notebook_launcher(training_function)

NameError: name 'training_function' is not defined

You can find more examples in the [🤗 Accelerate repo](https://github.com/huggingface/accelerate/tree/main/examples) repo.

<b>Extras</b>

In [51]:
# Check if PyTorch is using the GPU?
print (torch.cuda.is_available())
print (torch.cuda.device_count())
print (torch.cuda.current_device()) # 0 if GPU Device 0

False
0


AssertionError: Torch not compiled with CUDA enabled

# Fine-tuning, Check!

That was fun! <b>In the first two chapters you learned about models and tokenizers</b>, and <b>now you know how to fine-tune them for your own data</b>. To recap, in this chapter you:

- Learned about datasets in the Hub
- Learned how to load and preprocess datasets, including using dynamic padding and collators
- Implemented your own fine-tuning and evaluation of a model
- Implemented a lower-level training loop
- Used 🤗 Accelerate to easily adapt your training loop so it works for multiple GPUs or TPUs
    - Did not try this as I do not have access to distributed system (multiple GPUs or TPUs)